# X4: Metastability / Survival Test (NLP)

**Paper-A (P0009) — X Series: Cross-Domain Generality**

| Item | Value |
|------|-------|
| Dataset | SST-2 (binary sentiment) |
| Model | DistilBERT (pretrained) |
| Noise | Symmetric, η = 0.8 |
| Trusted ratio | 5% |
| λ values | **0.34** (bad side), **0.36** (boundary), **0.42** (good side, control) |
| Epochs | **12** (4× baseline, long survival test) |
| Seeds | 0–29 (n=30 per λ) |
| Total runs | **90** |
| Estimated time | ~3–4 hours (A100) |

**Purpose**: Test how long a "good" learning trajectory survives  
when λ is set near or slightly below the phase boundary (λ≈0.37).

**Motivation**: X2 established the boundary at λ≈0.37. X3-fast showed  
irreversible trapping. X4 asks: at boundary-adjacent λ, do some runs  
stay "good" while others collapse? How does survival fraction decay over epochs?


## 1. Setup

In [ ]:
!pip install -q transformers datasets accelerate

import os
import json
import time
import copy
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification
)
from datasets import load_dataset

warnings.filterwarnings("ignore")
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} ({gpu.total_mem/1e9:.1f} GB)" if hasattr(gpu, 'total_mem') else f"GPU: {gpu.name}")


## 2. Configuration

In [ ]:
CONFIG = {
    "experiment": "exp_X4_metastability",
    "series": "X",
    "experiment_id_prefix": "X4",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "distilbert-base-uncased",
    "noise_type": "symmetric",
    "noise_rate": 0.8,
    "trusted_ratio": 0.05,
    "lambda_values": [0.34, 0.36, 0.42],
    "seeds": list(range(30)),
    "epochs": 12,
    "batch_size": 64,
    "learning_rate": 2e-5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "max_seq_length": 128,
    "use_bf16": True,
    "max_train_samples": 16000,
    "csv_measure_every_n_steps": 25,
    "output_dir": "X4_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
steps_per_epoch = n_used // CONFIG["batch_size"]
total_steps = steps_per_epoch * CONFIG["epochs"]
n_runs = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
est_per_run = total_steps * 25 / 1000 + 15

print(f"Experiment: {CONFIG['experiment']}")
print(f"ID prefix: {CONFIG['experiment_id_prefix']}")
print(f"Total runs: {n_runs} ({len(CONFIG['lambda_values'])} λ × {len(CONFIG['seeds'])} seeds)")
print(f"λ values: {CONFIG['lambda_values']}")
print(f"Epochs: {CONFIG['epochs']} (4× baseline)")
print(f"Steps/epoch: ~{steps_per_epoch}, Total steps/run: ~{total_steps}")
print(f"Est. time/run: ~{est_per_run:.0f}s")
print(f"Est. total: ~{n_runs * est_per_run / 3600:.1f}h")


## 3. Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Paper-A_X_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/{CONFIG['experiment']}"
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
LOCAL_OUTPUT = CONFIG["output_dir"]

def save_to_drive(filename, data):
    """Save JSON to both local and Drive."""
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        path = os.path.join(d, filename)
        with open(path, "w") as f:
            json.dump(data, f, indent=2)

def save_figure_to_drive(fig, filename, dpi=300):
    """Save figure to both local and Drive."""
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        fig.savefig(os.path.join(d, filename), dpi=dpi, bbox_inches="tight", facecolor="white")

print(f"Local: {LOCAL_OUTPUT}")
print(f"Drive: {DRIVE_OUTPUT}")


## 4. Deterministic Seeding

In [ ]:
def set_seed(seed):
    """Set all random seeds for full reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## 5. Data Preparation

In [ ]:
print("Loading SST-2...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
raw_dataset = load_dataset("glue", "sst2")

train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")

print("Tokenizing...")
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length",
                            max_length=CONFIG["max_seq_length"], return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length",
                           max_length=CONFIG["max_seq_length"], return_tensors="pt")
print("Done.")


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

test_dataset = TextDataset(test_encodings, torch.tensor(test_labels, dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print(f"Test loader: {len(test_dataset)} samples")


## 6. Noise Injection & Trusted Subset

In [ ]:
def prepare_noisy_and_trusted(train_labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    rng = np.random.RandomState(seed + 10000)
    n = len(train_labels)
    labels = np.array(train_labels)
    indices = np.arange(n)

    # Stratified trusted selection
    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels == c]
        n_sel = max(1, int(len(class_idx) * trusted_ratio))
        trusted_indices.extend(rng.choice(class_idx, size=n_sel, replace=False))
    trusted_indices = np.array(sorted(trusted_indices))

    # Noisy pool
    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    # Flip labels
    noisy_labels = labels.copy()
    n_flip = int(len(noisy_indices) * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        candidates = [c for c in range(num_classes) if c != noisy_labels[idx]]
        noisy_labels[idx] = rng.choice(candidates)

    # Subsample noisy pool
    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = np.sort(rng.choice(noisy_indices, size=max_noisy_samples, replace=False))

    actual = np.mean(noisy_labels[noisy_indices] != labels[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} | Noisy: {len(noisy_indices)} (η={actual:.3f})")
    return torch.tensor(noisy_labels, dtype=torch.long), trusted_indices, noisy_indices

# Quick test
print("Noise test:")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.8, 0.05, 2, 0, 16000)


## 7. Dual-Gradient Core

In [ ]:
def get_head_param_names(model):
    """Return parameter names of the classification head."""
    return [n for n, _ in model.named_parameters()
            if "classifier" in n or "pre_classifier" in n]

def extract_grads(model):
    """Extract gradients from all parameters."""
    return {n: p.grad.clone() for n, p in model.named_parameters() if p.grad is not None}

def normalize_grad_dict(grads):
    """Normalize gradient dictionary to unit norm."""
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0

def cosine_sim(grads_a, grads_b, param_names=None):
    """Cosine similarity between two gradient dictionaries."""
    keys = [n for n in (param_names or sorted(set(grads_a) & set(grads_b)))
            if n in grads_a and n in grads_b]
    if not keys: return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()

def set_mixed_grad(model, g_struct_n, g_value_n, lam):
    """Set mixed gradient on model parameters."""
    for n, p in model.named_parameters():
        if n in g_struct_n and n in g_value_n:
            p.grad = (1 - lam) * g_struct_n[n] + lam * g_value_n[n]
        elif n in g_struct_n:
            p.grad = (1 - lam) * g_struct_n[n]
        elif n in g_value_n:
            p.grad = lam * g_value_n[n]

@torch.no_grad()
def evaluate(model, test_loader, device):
    """Evaluate model on test set."""
    model.eval()
    correct = total = 0
    total_loss = 0.0
    n = 0
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        preds = out.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += out.loss.item()
        n += 1
    model.train()
    acc = correct / total
    return acc, 1.0 - acc, total_loss / n

print("Core functions defined.")


## 8. Single Run (Epoch-Level Logging)

In [ ]:
def run_single_experiment(lam, seed, config):
    """
    Execute one dual-gradient training run with epoch-level logging.
    Returns per-epoch test error for survival curve construction.
    """
    exp_id = f"{config['experiment_id_prefix']}-{seed:03d}-lam{lam:.2f}"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    # Data preparation
    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed, config.get("max_train_samples")
    )

    noisy_dataset = TextDataset(
        {k: v[noisy_idx] for k, v in train_encodings.items()},
        noisy_labels[noisy_idx]
    )
    trusted_dataset = TextDataset(
        {k: v[trusted_idx] for k, v in train_encodings.items()},
        noisy_labels[trusted_idx]
    )

    struct_loader = DataLoader(noisy_dataset, batch_size=config["batch_size"],
                               shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_dataset,
                              batch_size=min(config["batch_size"], len(trusted_dataset)),
                              shuffle=True, drop_last=False)

    # Model
    set_seed(seed)
    model = DistilBertForSequenceClassification.from_pretrained(
        config["model_name"], num_labels=config["num_classes"]
    ).to(device)
    model.train()
    head_params = get_head_param_names(model)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32
    measure_every = config.get("csv_measure_every_n_steps", 25)

    # Training with epoch logging
    epoch_logs = []
    global_step = 0
    best_error = 1.0
    best_epoch = 0

    for epoch in range(config["epochs"]):
        value_iter = iter(value_loader)
        epoch_csv = []
        epoch_loss_s = 0.0
        epoch_loss_v = 0.0
        n_batches = 0

        for struct_batch in struct_loader:
            # Struct gradient
            optimizer.zero_grad()
            s_ids = struct_batch["input_ids"].to(device)
            s_mask = struct_batch["attention_mask"].to(device)
            s_labels = struct_batch["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_s = model(input_ids=s_ids, attention_mask=s_mask, labels=s_labels)
            out_s.loss.backward()
            g_struct = extract_grads(model)

            # Value gradient
            optimizer.zero_grad()
            try:
                vb = next(value_iter)
            except StopIteration:
                value_iter = iter(value_loader)
                vb = next(value_iter)
            v_ids = vb["input_ids"].to(device)
            v_mask = vb["attention_mask"].to(device)
            v_labels = vb["labels"].to(device)
            with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                out_v = model(input_ids=v_ids, attention_mask=v_mask, labels=v_labels)
            out_v.loss.backward()
            g_value = extract_grads(model)

            # Mix and step
            g_sn, _ = normalize_grad_dict(g_struct)
            g_vn, _ = normalize_grad_dict(g_value)
            set_mixed_grad(model, g_sn, g_vn, lam)
            optimizer.step()

            global_step += 1
            n_batches += 1
            epoch_loss_s += out_s.loss.item()
            epoch_loss_v += out_v.loss.item()

            if global_step % measure_every == 0:
                c = cosine_sim(g_struct, g_value, head_params)
                epoch_csv.append(c)

        # End-of-epoch evaluation
        test_acc, test_error, test_loss = evaluate(model, test_loader, device)

        if test_error < best_error:
            best_error = test_error
            best_epoch = epoch + 1

        epoch_log = {
            "epoch": epoch + 1,
            "test_acc": round(test_acc, 4),
            "test_error": round(test_error, 4),
            "test_loss": round(test_loss, 4),
            "loss_struct": round(epoch_loss_s / max(n_batches, 1), 4),
            "loss_value": round(epoch_loss_v / max(n_batches, 1), 4),
            "c_sv_head": round(float(np.mean(epoch_csv)) if epoch_csv else 0.0, 4),
        }
        epoch_logs.append(epoch_log)

    elapsed = time.time() - t_start
    final_acc, final_error, _ = evaluate(model, test_loader, device)

    result = {
        "experiment_id": exp_id,
        "experiment": config["experiment"],
        "lambda": lam,
        "seed": seed,
        "test_acc": round(final_acc, 4),
        "test_error": round(final_error, 4),
        "best_error": round(best_error, 4),
        "best_epoch": best_epoch,
        "epoch_logs": epoch_logs,
        "avg_cos_struct_value": round(float(np.mean([l["c_sv_head"] for l in epoch_logs])), 4),
        "total_steps": global_step,
        "time_seconds": round(elapsed, 1),
    }

    del model, optimizer
    torch.cuda.empty_cache()
    return result

print("Single experiment function defined.")


## 9. Main Execution Loop

In [ ]:
all_results = []
n_total = len(CONFIG["lambda_values"]) * len(CONFIG["seeds"])
run_count = 0
t_total = time.time()

print("=" * 70)
print(f"X4: Metastability Test — {n_total} runs")
print(f"  λ = {CONFIG['lambda_values']}")
print(f"  Seeds: {len(CONFIG['seeds'])}, Epochs: {CONFIG['epochs']}")
print("=" * 70)

for lam in CONFIG["lambda_values"]:
    lam_results = []
    print(f"\n{'='*50}")
    print(f"λ = {lam:.2f} ({len(CONFIG['seeds'])} seeds × {CONFIG['epochs']} epochs)")
    print(f"{'='*50}")

    for seed in CONFIG["seeds"]:
        run_count += 1
        print(f"  [{run_count}/{n_total}] seed={seed}", end=" → ")
        result = run_single_experiment(lam, seed, CONFIG)
        all_results.append(result)
        lam_results.append(result)

        # Progress
        elapsed = time.time() - t_total
        eta = elapsed / run_count * (n_total - run_count)
        print(f"err={result['test_error']:.4f} best={result['best_error']:.4f}@ep{result['best_epoch']} "
              f"({result['time_seconds']:.0f}s) [ETA: {eta/60:.0f}m]")

    # Save per-λ results incrementally
    save_to_drive(f"X4_lambda{lam:.2f}_results.json", lam_results)
    
    # Quick summary for this λ
    errors = [r["test_error"] for r in lam_results]
    bests = [r["best_error"] for r in lam_results]
    print(f"  Summary λ={lam:.2f}: final={np.mean(errors):.4f}±{np.std(errors):.4f} "
          f"best={np.mean(bests):.4f}±{np.std(bests):.4f}")

# Save everything
total_time = time.time() - t_total
save_to_drive("X4_results.json", all_results)
save_to_drive("X4_config.json", CONFIG)

print(f"\n{'='*70}")
print(f"X4 COMPLETE: {n_total} runs in {total_time/60:.1f} min ({total_time/3600:.1f} h)")
print(f"{'='*70}")


## 10. Survival Curve Analysis

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Load results
path = os.path.join(LOCAL_OUTPUT, "X4_results.json")
if not os.path.exists(path):
    path = os.path.join(DRIVE_OUTPUT, "X4_results.json")
with open(path) as f:
    results = json.load(f)

# Organize by λ
lambda_data = {}
for r in results:
    lam = r["lambda"]
    if lam not in lambda_data:
        lambda_data[lam] = []
    lambda_data[lam].append(r)

lambdas = sorted(lambda_data.keys())
n_epochs = CONFIG["epochs"]

# ============================================================
# Define "good" threshold: error < 0.50 (better than random)
# ============================================================
GOOD_THRESHOLD = 0.50

print("=" * 60)
print(f"X4: Survival Analysis (threshold: error < {GOOD_THRESHOLD})")
print("=" * 60)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# --- (A) Survival curve ---
ax = axes[0, 0]
colors = {"0.34": "#D62728", "0.36": "#FF7F0E", "0.42": "#2CA02C"}

for lam in lambdas:
    runs = lambda_data[lam]
    n_runs = len(runs)
    survival = []
    
    for epoch in range(1, n_epochs + 1):
        n_good = sum(1 for r in runs if r["epoch_logs"][epoch-1]["test_error"] < GOOD_THRESHOLD)
        survival.append(n_good / n_runs)
    
    color = colors.get(f"{lam:.2f}", "#333333")
    ax.plot(range(1, n_epochs+1), survival, 'o-', color=color,
            linewidth=2.5, markersize=7, label=f"λ={lam:.2f}")
    ax.fill_between(range(1, n_epochs+1), survival, alpha=0.1, color=color)

ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.set_xlabel("Epoch", fontsize=13)
ax.set_ylabel(f"Fraction with error < {GOOD_THRESHOLD}", fontsize=13)
ax.set_title(f"(A) Survival Curve (threshold = {GOOD_THRESHOLD})", fontsize=13, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.05, 1.05)
ax.set_xlim(0.5, n_epochs + 0.5)

# --- (B) Epoch trajectories (mean ± std) ---
ax = axes[0, 1]
for lam in lambdas:
    runs = lambda_data[lam]
    trajs = np.array([[l["test_error"] for l in r["epoch_logs"]] for r in runs])
    means = trajs.mean(axis=0)
    stds = trajs.std(axis=0)
    epochs = range(1, n_epochs+1)
    color = colors.get(f"{lam:.2f}", "#333333")
    ax.plot(epochs, means, 'o-', color=color, linewidth=2.5, markersize=6, label=f"λ={lam:.2f}")
    ax.fill_between(epochs, means-stds, means+stds, alpha=0.15, color=color)

ax.axhline(0.5, color="gray", ls=":", alpha=0.4, label="Random chance")
ax.set_xlabel("Epoch", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(B) Error Trajectory (mean ± std)", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# --- (C) Individual traces ---
ax = axes[1, 0]
for lam in lambdas:
    runs = lambda_data[lam]
    color = colors.get(f"{lam:.2f}", "#333333")
    for r in runs:
        errs = [l["test_error"] for l in r["epoch_logs"]]
        ax.plot(range(1, n_epochs+1), errs, '-', color=color, alpha=0.15, linewidth=0.8)
    # Mean overlay
    trajs = np.array([[l["test_error"] for l in r["epoch_logs"]] for r in runs])
    ax.plot(range(1, n_epochs+1), trajs.mean(axis=0), 'o-', color=color,
            linewidth=3, markersize=6, label=f"λ={lam:.2f} mean", zorder=5)

ax.axhline(GOOD_THRESHOLD, color="black", ls="--", alpha=0.5, label=f"Threshold ({GOOD_THRESHOLD})")
ax.set_xlabel("Epoch", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(C) Individual Run Traces", fontsize=13, fontweight="bold")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# --- (D) Final vs Best error scatter ---
ax = axes[1, 1]
for lam in lambdas:
    runs = lambda_data[lam]
    finals = [r["test_error"] for r in runs]
    bests = [r["best_error"] for r in runs]
    color = colors.get(f"{lam:.2f}", "#333333")
    ax.scatter(bests, finals, color=color, alpha=0.6, s=50, edgecolors="black",
               linewidth=0.5, label=f"λ={lam:.2f}")

# Diagonal
lim = [0, 1]
ax.plot(lim, lim, 'k--', alpha=0.3, label="No degradation")
ax.set_xlabel("Best Error (across epochs)", fontsize=13)
ax.set_ylabel("Final Error (epoch 12)", fontsize=13)
ax.set_title("(D) Degradation: Final vs Best", fontsize=13, fontweight="bold")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")

fig.suptitle("X4: Metastability — SST-2 / DistilBERT / η=0.8 / 12 epochs",
             fontsize=15, fontweight="bold", y=1.02)
plt.tight_layout()
save_figure_to_drive(fig, "X4_metastability.png")
plt.show()

# ============================================================
# Summary statistics
# ============================================================
print("\n" + "=" * 60)
print("X4 SUMMARY")
print("=" * 60)
for lam in lambdas:
    runs = lambda_data[lam]
    finals = [r["test_error"] for r in runs]
    bests = [r["best_error"] for r in runs]
    best_epochs = [r["best_epoch"] for r in runs]
    n_survive = sum(1 for e in finals if e < GOOD_THRESHOLD)
    n_ever_good = sum(1 for b in bests if b < GOOD_THRESHOLD)
    degradation = [f - b for f, b in zip(finals, bests)]
    
    print(f"\nλ = {lam:.2f} ({len(runs)} runs):")
    print(f"  Final error:  {np.mean(finals):.4f} ± {np.std(finals):.4f}")
    print(f"  Best error:   {np.mean(bests):.4f} ± {np.std(bests):.4f}")
    print(f"  Best epoch:   {np.mean(best_epochs):.1f} ± {np.std(best_epochs):.1f}")
    print(f"  Degradation:  {np.mean(degradation):.4f} ± {np.std(degradation):.4f}")
    print(f"  Ever good:    {n_ever_good}/{len(runs)} ({100*n_ever_good/len(runs):.0f}%)")
    print(f"  Survive ep12: {n_survive}/{len(runs)} ({100*n_survive/len(runs):.0f}%)")


## 11. Download Results

In [ ]:
print(f"All results saved to: {DRIVE_OUTPUT}")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    sz = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f} ({sz/1024:.1f} KB)")

import shutil
shutil.make_archive("X4_results", "zip", LOCAL_OUTPUT)
try:
    from google.colab import files
    files.download("X4_results.zip")
except:
    pass
